### WARNING: For this function to work you have to manually delete the _**two** empty newlines_ at the end of the raw meassurement files, else the for loop will exit abnormally (it'll think there is one extra block).

In [49]:
def meass2df(meass_path):
    # Open the file and separate it in blocks (each block is delimited by an empty newline)
    with open(meass_path, 'r') as meass:
        blocks = meass.read().split("\n\n")
    
    # initialize empty arrays that we will give to the data frame
    width = []
    fps = []
    num_psr = []
    pcpu_grp = []

    for block in blocks:
        pcpu = []
        s = set()
        # blocks are still a single line, we split them into single lines
        lines = block.split("\n")
        # I made the processes first write the frame width and then fps so it
        # is safe to assume they will always be the first two lines
        width.append(int(lines[0]))
        fps.append(float(lines[1]))
        # we look for the lines that are relevant for the desired meassurement
        # and append the necessary data
        for line in lines[2:]:
            elems = line.split()
            if elems[0].find("Eye") != -1:
                s.add(elems[-2])
                pcpu.append(float(elems[-1]))
        num_psr.append(len(s))
        pcpu_grp.append(pcpu)

    # create the data frame from the "labeled" lists
    df = pd.DataFrame({"width" : width,
                          "fps" : fps,
                          "num_psrs" : num_psr,
                          "cpu_p1" : [x[0] for x in pcpu_grp],
                         "cpu_p2" : [x[1] for x in pcpu_grp],
                         "cpu_p3" : [x[2] for x in pcpu_grp],
                         "cpu_p4" : [x[3] for x in pcpu_grp],
                         "cpu_p5" : [x[4] for x in pcpu_grp]})

    return df

In [50]:
# Change file paths accordingly
df_eye = meass2df("/home/dv/Documents/Uni/KAIST/CDSN/Misc/EyeDet_stats")
df_face = meass2df("/home/dv/Documents/Uni/KAIST/CDSN/Misc/FaceDet_stats")

In [51]:
# Optionally save to csv
df_eye.to_csv("/home/dv/Documents/Uni/KAIST/CDSN/Misc/EyeDet_meass_raw.csv")
df_face.to_csv("/home/dv/Documents/Uni/KAIST/CDSN/Misc/FaceDet_meass_raw.csv")

In [64]:
width_grp_eye = df_eye.groupby("width")
width_grp_face = df_face.groupby("width")

In [59]:
df_eye["cpu_tot"] = df_eye["cpu_p1"] + df_eye["cpu_p2"] + df_eye["cpu_p3"] + df_eye["cpu_p4"] + df_eye["cpu_p5"]

In [61]:
df_face["cpu_tot"] = df_face["cpu_p1"] + df_face["cpu_p2"] + df_face["cpu_p3"] + df_face["cpu_p4"] + df_face["cpu_p5"]

In [69]:
df_data = []
idx_1 = []
idx_2 = []
for label, groups in zip(["Eye", "Face"], [width_grp_eye, width_grp_face]):
    for key, data in groups:
        row = []
        avg_pcpu = data["cpu_tot"].mean()
        avg_psr = data["num_psrs"].mean()
        avg_cpu_psr = avg_pcpu / avg_psr
        avg_fps = data["fps"].mean()
        # create levels for MultiIndex
        idx_1.append(label)
        idx_2.append(str(key))
        # Append row data for sublevel
        row.append(avg_fps)
        row.append(avg_pcpu)
        row.append(avg_cpu_psr)
        row.append(avg_psr)
        df_data.append(row)
        # print("AvgCPU: {cpu}\nAvgCPUperPSR: {cpu_psr}\nAvgFPS: {fps}\n".format(cpu=avg_pcpu, cpu_psr=avg_cpu_psr, fps=avg_fps))

arrays = [idx_1, idx_2]    
df_all = pd.DataFrame(df_data, index=arrays, columns=["AvgFPS", "AvgCPU", "AvgCPU/PSR", "AvgPSR"])

The first index refers to the haarcascade classifier used. The second index refers to the _width_ of the scaled frame. The height is calculated according to the ratio of the original resolution. The ratio is usually $1.\overline{33}:1.00$. An example is w=640 h=480, $\frac{w}{h} = 1.33$

In [70]:
df_all

AvgFPS      AvgCPU  AvgCPU/PSR    AvgPSR
Eye  250   8.233171  129.242857   37.307216  3.464286
     300   5.866057  134.060714   39.101042  3.428571
     350   4.256630  139.796296   42.892045  3.259259
     400   2.701828  143.246154   40.927473  3.500000
     450   2.025430  145.130435   44.506667  3.260870
     500   1.658889  148.180000   41.623596  3.560000
     550   1.310821  149.297222   47.146491  3.166667
Face 250  11.656264  129.610714   36.291000  3.571429
     300   8.102803  134.341379   39.352525  3.413793
     350   6.415242  140.078571   40.856250  3.428571
     400   4.288500  143.381481   42.079348  3.407407
     450   3.540669  145.353846   42.945455  3.384615
     500   2.351676  148.233333   43.385366  3.416667
     550   1.966843  149.308696   44.310968  3.369565

In [71]:
df_all.to_csv("/home/dv/Documents/Uni/KAIST/CDSN/Misc/multiapp_meass.csv")

In [72]:
df_all.to_latex("/home/dv/Documents/Uni/KAIST/CDSN/Misc/multiapp_meass.tex")